In [54]:
import pandas as pd
import requests
from fuzzywuzzy import fuzz
from bs4 import BeautifulSoup, Comment
import time
import numpy as np

In [67]:
kpop_bbh100 = pd.read_csv('Kpop on BBH100 - filled.csv')
kpop_bbh100.dropna(how='all', inplace=True)
kpop_bbh100.dropna(axis=1, how='all', inplace=True)
kpop_bbh100.insert(4, 'Original Lyrics Text', ['' for i in range(len(kpop_bbh100))])
kpop_bbh100.insert(6, 'Translated Lyrics Text', ['' for i in range(len(kpop_bbh100))])
kpop_bbh100.head()

,Chart date[16],Artist,Song/Album Title,Original URL,Original Lyrics Text,English Translation URL,Translated Lyrics Text,Peak position,Weeks on Chart
0,2009-10-31,Wonder Girls,"""Nobody (English release)""[17]",https://genius.com/Wonder-girls-nobody-english...,,https://genius.com/Wonder-girls-nobody-english...,,76,1
1,2012-09-22,Psy[18],"""Gangnam Style""[19][20]",https://genius.com/Psy-gangnam-style-lyrics,,https://genius.com/Genius-english-translations...,,2 (Peaked at #2 on 2012-10-06 for 7 weeks),31
2,2013-04-27,Psy,"""Gentleman""",https://genius.com/Psy-gentleman-lyrics,,https://lyricstranslate.com/en/gentleman-gentl...,,5,15
3,2014-06-28,Psy feat. Snoop Dogg,"""Hangover""",https://genius.com/Psy-hangover-lyrics,,https://colorcodedlyrics.com/2014/06/psy-ssai-...,,26,1
4,2015-12-19,Psy feat. CL[21],"""Daddy""",https://genius.com/Psy-daddy-lyrics,,https://lyricstranslate.com/en/daddy-daddy.html-1,,97,1


In [68]:
##standardize and process chart data

#remove wiki citations, standardize whitespace, remove quotation marks
#regex patterns and replacement strings
regex_pats = [[r"(\[\d+\])", ""], [r"(\s)", " "], [r"(\")", ""]]
for pat in regex_pats:
    kpop_bbh100["Artist"] = kpop_bbh100["Artist"].str.replace(pat[0], pat[1])
    kpop_bbh100["Song/Album Title"] = kpop_bbh100["Song/Album Title"].str.replace(pat[0], pat[1])
    kpop_bbh100["Peak position"] = kpop_bbh100["Peak position"].str.replace(pat[0], pat[1])

In [69]:
def scrape_genius_lyrics(url):
    #if it isn't a genius url, just return empty string; will fill later
    if 'genius.com' not in url:
        print('ERROR: NOT A VALID GENIUS URL')
        return ""

    #get page content
    song_page = requests.get(url)
    song_soup = BeautifulSoup(song_page.content, 'html.parser')

    ##SONG LYRICS
    #sometimes the full page content does not load on the first try
    #soooooo this code re-calls the song url until it acquires the desired info :) 
    #will do this 8 times before giving up to prevent infinite code
    #it normally works after ~2 attempts
    lyrics_i = 0
    song_lyrics = url
    while song_lyrics == url and lyrics_i < 8:
        lyrics_i += 1
        try:
            ##collect song lyrics and store as string (ugly string, but still)
            #finding the div stopped working for some reason so will do this for now
            #if lyrics div is present, this will find it :D
            song_lyrics = song_soup.find("div", "lyrics").get_text()
        except: 
            #re-access web page
            song_page = requests.get(s)
            time.sleep(6) #sleep to be nice
            song_soup = BeautifulSoup(song_page.content, 'html.parser')
    #if it gave up raise error
    if song_lyrics == url and lyrics_i >= 8:
        #problematic_urls.append(s)  
        print(lyrics_i, "lyrics error: limit exceeded", s)

    #return lyrics string
    return song_lyrics

In [71]:
for i, r in kpop_bbh100.iterrows():
    #scrape url for original lyrics + store
    original_url = r['Original URL']
    lyrics = scrape_genius_lyrics(original_url)
    kpop_bbh100.at[i, 'Original Lyrics Text'] = lyrics
    print(i, lyrics[:20])
    time.sleep(6) #sleep to be nice

0 

You know I still l
1 

Hangul

[Intro]
오빤
2 

[Verse 1]
알랑가몰라 왜 
3 

[Chorus]
Hangover,
4 

[Chorus: PSY]
I go
5 

[Intro]
Hey, you, 
6 

[방탄소년단의 DNA 가사]

[
7 

[Intro: Desiigner]
8 

[방탄소년단의 FAKE LOVE 
9 

[블랙핑크 "뚜두뚜두" 가사]


10 

[방탄소년단 X 니키미나즈의 ID
11 

[Verse 1: Dua Lipa
12 

[Verse 1: Jungkook
13 

[블랙핑크 "Kill This L
14 

[방탄소년단 작은 것들을 위한 시
15 

[Verse 1: Lauv]
Ye
16 

[Intro: Becky G]
I
17 

[Intro: Jungkook]

18 

[방탄소년단 - ON 가사]

[
19 

[방탄소년단 - 시차 (My Ti
20 

[방탄소년단 - Filter 가사
21 

[대취타 (Daechwita) 가
22 

[Intro: Jennie]
So
23 

[Intro]
BTS

[Chor
24 

[블랙핑크 "How You Lik
25 

[Intro: Jungkook]

26 

[Verse 1: Selena G
27 

[블랙핑크 "Lovesick Gi
28 

[방탄소년단 "Life Goes 
29 

[방탄소년단 "Blue & Gre
30 

[방탄소년단 "Stay" 가사]

31 

[방탄소년단 "내 방을 여행하는 
32 

[방탄소년단 "잠시" 가사]

[
33 

[방탄소년단 "병" 가사]

[I


In [70]:
for i, r in kpop_bbh100.iterrows():
    #scrape url for translated lyrics + store
    original_url = r['English Translation URL']
    lyrics = scrape_genius_lyrics(original_url)
    kpop_bbh100.at[i, 'Translated Lyrics Text'] = lyrics
    print(i, lyrics[:20])
    time.sleep(6) #sleep to be nice

0 

You know I still l
1 

[Intro]
Oppa Gangn
ERROR: NOT A VALID GENIUS URL
2 
ERROR: NOT A VALID GENIUS URL
3 
ERROR: NOT A VALID GENIUS URL
4 
5 

[Intro]
Hey, you, 
6 

[Verse 1: V]
At fi
ERROR: NOT A VALID GENIUS URL
7 
8 

[Intro: V, Jungkoo
9 

[Intro: All & Rosé
10 

[Intro: Nicki Mina
ERROR: NOT A VALID GENIUS URL
11 
12 

[Verse 1: Jungkook
13 

[Intro: Jennie & L
14 

[Intro]
(Oh-oh-oh-
15 

[Verse 1: Lauv]
Ye
16 

[Intro: Becky G]
I
17 

[Intro: Jungkook]

18 

[Verse 1: Jimin, V
19 

[Verse 1: Jungkook
20 

[Verse 1: Jimin]
T
21 

[Sample: National 
22 

[Intro: Jennie]
So
ERROR: NOT A VALID GENIUS URL
23 
24 

[Intro: Lisa]
BLAC
25 

[Intro: Jungkook]

26 

[Verse 1: Selena G
27 

[Intro: All]
(Love
28 

[Intro]
I remember
29 

[Intro: V, Jung Ko
30 

[Verse 1: Jung Koo
31 

[Intro: Jimin, V]

32 

[Intro: Jung Kook,
33 

[Intro: j-hope]
Mm


In [77]:
#fill in translated lyrics not in genius
for i, r in kpop_bbh100.iterrows():
    if r["Translated Lyrics Text"] == "":
        title = r["Song/Album Title"]
        lyrics = open("Manual Lyrics/"+title+".txt", "r").read()
        kpop_bbh100.at[i, 'Translated Lyrics Text'] = lyrics
        print(i, lyrics[:20])

2 [Verse 1]
I don’t kn
3 [Chorus]
Hangover, h
4 [Chorus: PSY]
I got 
7 [Intro: Desiigner]
G
11 [Verse 1: Dua Lipa]

23 [Intro]
BTS

[Chorus


In [79]:
kpop_bbh100.to_csv('BBH100_Complete_Kpop.csv')